In [17]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from torch.nn.utils.rnn import pad_sequence
import collections
from torch.utils.data import random_split
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

# Cấu hình thiết bị
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- 1. Xây dựng Vocabulary ---
class Vocabulary:
    def __init__(self, freq_threshold):
        self.itos = {0: "<PAD>", 1: "<SOS>", 2: "<EOS>", 3: "<UNK>"}
        self.stoi = {"<PAD>": 0, "<SOS>": 1, "<EOS>": 2, "<UNK>": 3}
        self.freq_threshold = freq_threshold

    def __len__(self):
        return len(self.itos)

    def build_vocabulary(self, sentence_list):
        frequencies = collections.Counter()
        idx = 4
        
        for sentence in sentence_list:
            for word in sentence.lower().split():
                frequencies[word] += 1
                
        for word, count in frequencies.items():
            if count >= self.freq_threshold:
                self.stoi[word] = idx
                self.itos[idx] = word
                idx += 1

    def numericalize(self, text):
        tokenized_text = text.lower().split()
        return [
            self.stoi[token] if token in self.stoi else self.stoi["<UNK>"]
            for token in tokenized_text
        ]

# --- 2. Xây dựng Dataset ---
class Flickr8kDataset(Dataset):
    def __init__(self, root_dir, captions_file, transform=None, freq_threshold=5):
        self.root_dir = root_dir
        self.df = pd.read_csv(captions_file)
        self.transform = transform
        
        # Lấy captions và ảnh
        self.imgs = self.df["image"]
        self.captions = self.df["caption"]
        
        # Xây dựng vocab
        self.vocab = Vocabulary(freq_threshold)
        self.vocab.build_vocabulary(self.captions.tolist())

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        caption = self.captions[index]
        img_id = self.imgs[index]
        img = Image.open(os.path.join(self.root_dir, img_id)).convert("RGB")

        if self.transform is not None:
            img = self.transform(img)

        numericalized_caption = [self.vocab.stoi["<SOS>"]]
        numericalized_caption += self.vocab.numericalize(caption)
        numericalized_caption.append(self.vocab.stoi["<EOS>"])

        return img, torch.tensor(numericalized_caption)

# --- 3. Collate Function (Padding) ---
class MyCollate:
    def __init__(self, pad_idx):
        self.pad_idx = pad_idx

    def __call__(self, batch):
        imgs = [item[0].unsqueeze(0) for item in batch]
        imgs = torch.cat(imgs, dim=0)
        targets = [item[1] for item in batch]
        targets = pad_sequence(targets, batch_first=True, padding_value=self.pad_idx)
        return imgs, targets

# --- Thiết lập Path và Loader ---
# Lưu ý: Cấu trúc thư mục trên Kaggle thường là /kaggle/input/flickr8k/Images và captions.txt
image_folder = '/kaggle/input/flickr8k/Images'
captions_file = '/kaggle/input/flickr8k/captions.txt'

transform = transforms.Compose([
    transforms.Resize((224, 224)), # ResNet chuẩn input 224
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

# Load dữ liệu (Cần đảm bảo file tồn tại, nếu không code sẽ báo lỗi)
try:
    dataset = Flickr8kDataset(image_folder, captions_file, transform=transform)
    pad_idx = dataset.vocab.stoi["<PAD>"]
    # train_loader = DataLoader(
    #     dataset=dataset,
    #     batch_size=32, # Batch size theo yêu cầu
    #     num_workers=2,
    #     shuffle=True,
    #     collate_fn=MyCollate(pad_idx=pad_idx)
    # )

    # 1. Xác định kích thước tập train và test (ví dụ: 90% train, 10% test)
    train_size = int(0.9 * len(dataset))
    test_size = len(dataset) - train_size
    
    # 2. Chia ngẫu nhiên dataset
    train_set, test_set = random_split(dataset, [train_size, test_size])
    
    # 3. Tạo lại Train Loader (nếu muốn train trên đúng tập train_set mới)
    train_loader = DataLoader(
        dataset=train_set,
        batch_size=32,
        num_workers=2,
        shuffle=True,
        collate_fn=MyCollate(pad_idx=pad_idx)
    )
    
    # 4. Tạo Test Loader (đây là biến bạn đang thiếu)
    test_loader = DataLoader(
        dataset=test_set,
        batch_size=32,
        num_workers=2,
        shuffle=False, # Test không cần shuffle
        collate_fn=MyCollate(pad_idx=pad_idx)
    )

    print(f"Train size: {len(train_set)}, Test size: {len(test_set)}")
    print(f"Vocab size: {len(dataset.vocab)}")
except Exception as e:
    print(f"Lỗi load data: {e}. Hãy kiểm tra lại đường dẫn file trên Kaggle.")

Using device: cuda
Train size: 36409, Test size: 4046
Vocab size: 3005


In [18]:
import torchvision.models as models

import torch
import torch.nn as nn
import torchvision
from torchvision.models import ViT_B_16_Weights

class ViTEncoder(nn.Module):
    def __init__(self, out_dim=512):
        super().__init__()
        # Load pre-trained ViT-B/16
        weights = ViT_B_16_Weights.DEFAULT
        self.vit = torchvision.models.vit_b_16(weights=weights)
        
        # ViT-B/16 có hidden_dim = 768. 
        # Ta cần chiếu về out_dim (512) để khớp với Decoder LSTM ở Module 1
        self.proj = nn.Linear(768, out_dim)
        
        # Freeze các tầng đầu của ViT để train nhanh hơn (tùy chọn)
        for param in self.vit.parameters():
            param.requires_grad = False 
        # Unfreeze projection layer
        for param in self.proj.parameters():
            param.requires_grad = True

    def forward(self, x):
        # x: B x 3 x 224 x 224
        
        # 1. Chuyển ảnh thành patch embeddings
        # Hàm _process_input thực hiện: Conv2d (chia patch) -> Flatten -> Transpose
        x = self.vit._process_input(x) 
        n = x.shape[0]

        # 2. Thêm Class Token (Learnable)
        batch_class_token = self.vit.class_token.expand(n, -1, -1)
        x = torch.cat([batch_class_token, x], dim=1) # B x (1 + 196) x 768 (với ảnh 224x224, patch 16 -> 14x14=196 patches)

        # 3. Cộng Positional Embedding và đi qua Encoder Layers
        x = self.vit.encoder(x) # B x 197 x 768

        # 4. Chiếu về chiều dữ liệu mong muốn
        x = self.proj(x) # B x 197 x 512

        # Tách CLS token và Patch tokens
        cls_token = x[:, 0]     # Global Vector (B x 512)
        patch_tokens = x[:, 1:] # Memory cho Attention (B x 196 x 512)
        
        return cls_token, patch_tokens

# --- Attention ---
class AdditiveAttention(nn.Module):
    def __init__(self, dim_q, dim_k, dim_h):
        super().__init__()
        self.Wq = nn.Linear(dim_q, dim_h)
        self.Wk = nn.Linear(dim_k, dim_h)
        self.v = nn.Linear(dim_h, 1)

    def forward(self, q, k, mask=None):
        # q: B x hidden_dim, k: B x L x mem_dim
        q_ = self.Wq(q).unsqueeze(1)    # B x 1 x dim_h
        k_ = self.Wk(k)                 # B x L x dim_h
        
        # Broadcasting q_ cộng với k_
        attn_energy = self.v(torch.tanh(q_ + k_)).squeeze(-1) # B x L
        
        if mask is not None:
            attn_energy = attn_energy.masked_fill(mask == 0, -1e9)
            
        alpha = torch.softmax(attn_energy, dim=-1) # B x L
        context = torch.bmm(alpha.unsqueeze(1), k).squeeze(1) # B x mem_dim
        
        return context, alpha

# --- Decoder ---
class LSTMAttnDecoder(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, mem_dim):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        self.lstm = nn.LSTM(emb_dim + mem_dim, hidden_dim, batch_first=True)
        self.attn = AdditiveAttention(dim_q=hidden_dim, dim_k=mem_dim, dim_h=256) # dim_h=256 theo bài
        self.fc = nn.Linear(hidden_dim, vocab_size)
        self.hidden_dim = hidden_dim

    def forward(self, captions, memory):
        # captions: B x T
        B, T = captions.shape
        embeddings = self.emb(captions) # B x T x Emb
        
        # Khởi tạo hidden state và cell state
        h = torch.zeros(1, B, self.hidden_dim, device=captions.device)
        c = torch.zeros(1, B, self.hidden_dim, device=captions.device)
        
        outputs = []
        # Loop qua từng time step (Teacher Forcing)
        for t in range(T):
            h_curr = h[-1] # Lấy layer cuối cùng
            
            # Tính attention context
            context, _ = self.attn(h_curr, memory)
            
            # Input cho LSTM: ghép embedding từ hiện tại + context vector
            lstm_input = torch.cat([embeddings[:, t, :], context], dim=1).unsqueeze(1) # B x 1 x (Emb+Mem)
            
            out, (h, c) = self.lstm(lstm_input, (h, c))
            
            logits = self.fc(out.squeeze(1))
            outputs.append(logits.unsqueeze(1))
            
        return torch.cat(outputs, dim=1)

    def generate_caption(self, memory, vocab, max_len=20):
        # Hàm dùng cho inference (tạo caption cho 1 ảnh)
        # memory: 1 x L x D
        batch_size = memory.size(0)
        h = torch.zeros(1, batch_size, self.hidden_dim, device=memory.device)
        c = torch.zeros(1, batch_size, self.hidden_dim, device=memory.device)
        
        # Bắt đầu bằng thẻ <SOS>
        input_word = torch.tensor([vocab.stoi["<SOS>"]], device=memory.device)
        
        captions = []
        attentions = []
        
        for _ in range(max_len):
            embed = self.emb(input_word) # 1 x Emb
            h_curr = h[-1]
            
            context, alpha = self.attn(h_curr, memory)
            attentions.append(alpha.cpu().detach())
            
            lstm_input = torch.cat([embed, context], dim=1).unsqueeze(1)
            out, (h, c) = self.lstm(lstm_input, (h, c))
            
            output = self.fc(out.squeeze(1))
            predicted_idx = output.argmax(1)
            
            captions.append(predicted_idx.item())
            
            if predicted_idx.item() == vocab.stoi["<EOS>"]:
                break
                
            input_word = predicted_idx
            
        return [vocab.itos[idx] for idx in captions], attentions

# --- Tổng hợp Model ---
class ViTLSTMVgModel(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        
    def forward(self, images, captions):
        _, memory = self.encoder(images)
        outputs = self.decoder(captions, memory)
        return outputs

In [19]:
# --- Cấu hình Hyperparameters ---
# vocab_size lấy từ dataset Module 1
d_model = 512 
nhead = 8
num_layers = 4 # Giảm xuống 4 để train nhanh hơn trên Kaggle
# Lưu ý: CNNEncoder output dim phải khớp với d_model của Transformer
# Nếu CNN ra 2048, cần projection layer trong Encoder về 512

# Khởi tạo
encoder = CNNEncoder(out_dim=d_model).to(device) # Sử dụng CNNEncoder từ Module 1
decoder = TransformerCaptionDecoder(vocab_size, d_model, nhead, num_layers).to(device)
model = ViTLSTMVgModel(encoder, decoder).to(device)

import torch.optim as optim

criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)
optimizer = optim.Adam(model.parameters(), lr=1e-4) # LR thấp hơn chút cho ViT stable

print("Bắt đầu huấn luyện ViT + LSTM...")
model.train()

num_epochs = 3
for epoch in range(num_epochs):
    epoch_loss = 0
    for idx, (imgs, captions) in enumerate(train_loader):
        imgs = imgs.to(device)
        captions = captions.to(device)
        
        # Forward
        outputs = model(imgs, captions[:, :-1])
        targets = captions[:, 1:]
        
        loss = criterion(outputs.reshape(-1, vocab_size), targets.reshape(-1))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        
        if idx % 50 == 0:
            print(f"Epoch {epoch+1}, Step {idx}, Loss: {loss.item():.4f}")

    print(f"End Epoch {epoch+1}, Avg Loss: {epoch_loss/len(train_loader):.4f}")

Bắt đầu huấn luyện ViT + LSTM...
Epoch 1, Step 0, Loss: 8.1721
Epoch 1, Step 50, Loss: 4.5419
Epoch 1, Step 100, Loss: 4.3601
Epoch 1, Step 150, Loss: 3.8829
Epoch 1, Step 200, Loss: 3.8910
Epoch 1, Step 250, Loss: 3.6199
Epoch 1, Step 300, Loss: 3.5239
Epoch 1, Step 350, Loss: 3.0121
Epoch 1, Step 400, Loss: 3.3674
Epoch 1, Step 450, Loss: 3.4003
Epoch 1, Step 500, Loss: 3.4828
Epoch 1, Step 550, Loss: 3.1448
Epoch 1, Step 600, Loss: 3.3945
Epoch 1, Step 650, Loss: 3.1044
Epoch 1, Step 700, Loss: 2.8090
Epoch 1, Step 750, Loss: 2.9782
Epoch 1, Step 800, Loss: 3.0682
Epoch 1, Step 850, Loss: 3.1245
Epoch 1, Step 900, Loss: 3.0733
Epoch 1, Step 950, Loss: 2.6174
Epoch 1, Step 1000, Loss: 2.8605
Epoch 1, Step 1050, Loss: 2.8225
Epoch 1, Step 1100, Loss: 2.6017
End Epoch 1, Avg Loss: 3.3715
Epoch 2, Step 0, Loss: 2.6317
Epoch 2, Step 50, Loss: 2.6602
Epoch 2, Step 100, Loss: 2.5668
Epoch 2, Step 150, Loss: 2.6231
Epoch 2, Step 200, Loss: 2.5983
Epoch 2, Step 250, Loss: 2.3616
Epoch 2, Ste

## **Giải đáp câu hỏi phân tích (5.5)**

### **(a) Lợi ích và chi phí của "Attention ở cả hai phía" (Encoder Self-Attention & Decoder Cross-Attention):**

    - Lợi ích (Chất lượng):

        Encoder (Self-Attention): Mỗi patch (ví dụ: góc ảnh chứa cái cây) có thể "nhìn" thấy và trao đổi thông tin với patch khác (ví dụ: góc ảnh chứa bầu trời) ngay lập tức. Điều này giúp ViT hiểu ngữ cảnh toàn cục (global context) tốt hơn CNN (CNN cần nhiều lớp mới nhìn bao quát được). Feature map đầu ra của ViT giàu ngữ nghĩa hơn.

        Decoder (Cross-Attention): Giúp LSTM chọn lọc thông tin từ kho ngữ nghĩa phong phú đó để sinh từ chính xác.

    - Chi phí (Tính toán):

        Rất nặng. Self-Attention trong Encoder có độ phức tạp O(L^2) (với L là số patch). Cross-Attention trong Decoder có độ phức tạp O(T⋅L) (với T là độ dài câu).

### **(b) Khi số patch L tăng (Ví dụ: ảnh to hơn hoặc patch size nhỏ đi):**

    - Tại Encoder (ViT): Chi phí tăng theo bình phương O(L^2).

        Ví dụ: Nếu giảm patch size từ 16 xuống 8 → số patch tăng gấp 4 → chi phí tính toán attention tăng gấp 16 lần. Đây là điểm yếu chí tử của ViT thuần.

    - Tại Decoder (LSTM+Attn): Chi phí tăng tuyến tính O(L).

        Decoder chỉ cần tính tích vô hướng với L vector memory mỗi bước. Việc tăng L không gây áp lực quá lớn cho Decoder so với Encoder.